In [36]:
import sys
import os
import numpy as np
import shutil

In [37]:
output_root = '/scratch/zb2113/DedalusData/2D/' # Replace this by the location where you want the output to be produced
exec_file = '/home/zb2113/Research-Dedalus/2d-AR=64/Template.py' # This should be the location of the python file. 

In [38]:
param_name = 'Qrad'

param_values = ['0.0014','0.0028','0.0042','0.0056','0.0070'] 
# param_values = ['0.0014']
#param_values = ['0.0006']

In [39]:
namelist = \
    "Lx, Lz = 64, 1\n"+\
    "Nx, Nz = 8192, 128\n"+\
    "Ra_M=5e6\n"+\
    "Prandtl = 1\n"+\
    "D_0 = 0\n"+\
    "D_H = 1/3\n"+\
    "M_0 = 0\n"+\
    "M_H = -1\n"+\
    "N_s2 = 4/3\n"+\
    "Qrad = 0.0002\n"+\
    "stop_sim_time = 2500.0\n"
exec(namelist)

In [40]:
script_start = [  \
    "#!/bin/bash\n",\
    "#SBATCH --nodes=8\n",\
    "#SBATCH --tasks-per-node=32\n",\
    "#SBATCH --cpus-per-task=1\n",\
    "#SBATCH --mem=64GB\n",\
    "#SBATCH --time=24:00:00\n",\
    "#SBATCH --job-name=MRBC_TR\n",\
    "#SBATCH --mail-type=ALL\n",\
    "#SBATCH --mail-user=zb2113@nyu.edu\n",\
    "#SBATCH --output=slurm_%j.out\n",
    "module purge\n"]

script_end = [ \
              "srun /scratch/work/public/singularity/run-dedalus-3.0.0a0.bash python exec.py"]


In [41]:
file_launch = output_root + 'launch_slurm.sh'
launcher = open(file_launch,'w')
for value in param_values:
    exec(param_name + '=' + value)
    expname = 'MRBC_2D_RaM_'+"{:.1e}".format(Ra_M)  +'_Pr_'+"{:.1e}".format(Prandtl) \
    +'_QR_'+"{:.1e}".format(Qrad) \
            +'_DH_'+"{:.1e}".format(D_H) \
            +  '_Lx_'+"{:.1e}".format(Lx) \
            +   '_Nz_'+"{:d}".format(Nz)
    exp_dir = output_root + expname
    if (not os.path.exists(exp_dir)):
        os.makedirs(exp_dir, exist_ok = True)
    os.chdir(exp_dir)
    shutil.copyfile(exec_file,exp_dir+'/exec.py')
    nml_filename = 'MRBC2D_param.py'
    nml_file = open(nml_filename, 'w')
    nml_file.writelines(namelist)
    nml_file.writelines(param_name + '=' + value +'\n')
    nml_file.close()
    script_name = 'run_MRBC.slurm'
    script = open(script_name,'w')
    script.writelines(script_start)
    script.writelines('cd '+exp_dir+'\n')
    script.writelines(script_end)
    script.close()
    launcher.writelines("cd " + output_root + expname +'\n')
    launcher.writelines("sbatch ./" + script_name +'\n')
launcher.writelines("cd " + output_root + '\n')

launcher.close()
print('Jobs can be submitted to the queue by going to the directory:\n' + output_root)
print('and enter the command: \nsource launch_slurm.sh')


Jobs can be submitted to the queue by going to the directory:
/scratch/zb2113/DedalusData/2D/
and enter the command: 
source launch_slurm.sh


In [42]:
print(expname)

MRBC_2D_RaM_5.0e+06_Pr_1.0e+00_QR_7.0e-03_DH_3.3e-01_Lx_6.4e+01_Nz_128
